In [ ]:
%matplotlib inline

import tensorflow as tf
import pandas as pd
import nltk, re, time
from langdetect import detect
from contractions import get_contractions
from sqlalchemy import create_engine
from pprint import pprint
from nltk.corpus import stopwords
import chars2vec
import sklearn.decomposition
import matplotlib.pyplot as plt
import itertools
import string
import re
from keras.models import model_from_json
from keras.models import load_model
import gensim
import os
import numpy as np
from sklearn.utils import resample
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import roc_auc_score
from keras.initializers import Constant

# We download stopwords package
# nltk.download('stopwords')
contractions = get_contractions()


In [ ]:
# Load data
engine = create_engine('mysql://root:@localhost:3306/steam')
# steam_data_query = """SELECT url AS reviewid, content, CAST(recommend AS SIGNED) AS recommend
#     FROM latest_review"""
train_val_data_query = """SELECT gameid, url, CAST(recommend AS SIGNED) AS recommend, hours_2w, hours_all, posttime, updatetime, EAG, compensation, content, initial_release_date, lang 
FROM latest_review 
WHERE url NOT IN (SELECT DISTINCT url FROM test_set) 
AND lang = 'en';
"""

test_data_query = """SELECT gameid, url, CAST(recommend AS SIGNED) AS recommend, hours_2w, hours_all, posttime, updatetime, EAG, compensation, content, initial_release_date, lang
FROM test_set;"""

df_train_val = pd.read_sql(train_val_data_query, engine)
df_test = pd.read_sql(test_data_query, engine)


In [ ]:
def pre_process(data):
    review_lines = []

    lines = data['content'].values.tolist()


    for line in lines:
        tokens = word_tokenize(line)
        # convert to lower case
        tokens = [w.lower() for w in tokens]

        new_text = []
        for word in tokens:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)

        text = " ".join(new_text)
        # remove punctuation from each word    
    #     table = str.maketrans('', '', string.punctuation)
    #     stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
    #     words = [word for word in stripped if word.isalpha()]

        words = word_tokenize(text)
        # filter out stop words    
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        review_lines.append(words)
        
    return review_lines;


In [ ]:
print(len(review_lines))
skf = StratifiedKFold(n_splits = 5, shuffle = True)
num_folds = skf.get_n_splits(X)
for index, (train_index, val_index) in enumerate(skf.split(X, y)):
